This notebook validates the IMC models on the KGH (Kenema) dataset

In [ ]:
%matplotlib inline
%load_ext rpy2.ipython

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import r2_score
from sklearn.metrics import brier_score_loss
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

from scipy import interp
from scipy.interpolate import interp1d

In [ ]:
class LogRegModel(object):
    def __init__(self, fn, model_format='MICE'):
        self.intercept = 0
        self.names = []        
        self.terms = []
        if model_format == 'MICE':
            self.loadTermsMICE(fn)
        elif model_format == 'GLM':
            self.loadTermsGLM(fn)
            
    def setIntercept(self, b0):
        self.intercept = b0

    def addTerm(self, t):
        self.terms += [t]
        self.names += [t.name]        

    def linfeat(self, x):
        zmat = []
        for i in range(0, len(x)):
            xrow = x[i]
            zrow = [1.0]
            for j in range(0, len(self.terms)):
                t = self.terms[j]
                zrow += t.linearFeatures(xrow[j])
            zmat += [zrow]
        return zmat

    def lincoeff(self):
        coeff = [self.intercept]
        for t in self.terms:
            coeff += t.coeffs
        return coeff    
                
    def sigmoid(self, v):
        return 1.0 / (1.0 + np.exp(-v))
            
    def predict(self, x):
        z = self.linfeat(x)
        theta = self.lincoeff()
        prob = []
        n = len(z)
        for i in range(0, n):            
            p = self.sigmoid(np.dot(z[i], theta))
            prob += [p]
        return np.array(prob)

    def loatVarTypes(self, data_fn, dict_fn):
        var = []
        vtyp= []
        with open(data_fn) as f:
            var = f.readlines()[0].split(',')
        with open(dict_fn) as f:
            for line in f.readlines():
                line = line.strip()
                if not line: continue
                _, t = line.split(',')[0:2]
                vtyp += [t]
        for t in self.terms:
            pos = var.index(t.name)
            t.vtyp = vtyp[pos]
            
    def saveOddRatios(self, x, fn):
        theta = self.lincoeff()
        scale = [1.0] * len(theta)

        t = 0
        ts = 1
        for term in self.terms:
            vrang = term.varRanges(x[:,t]) 
            for i in range(0, len(vrang)):
                scale[ts] = vrang[i]
                if scale[ts] < 1: scale[ts] = 1.0 / scale[ts]
                ts = ts + 1                
            t = t + 1

        theta *= np.array(scale)
        odds = np.exp(theta)
        ts = 1
        with open(fn, 'w') as f:                
            for term in self.terms:
                vnam = term.varNames()
                for i in range(0, len(vnam)):
                    f.write(vnam[i] + ' ' + str(odds[ts]) + '\n')
                    ts = ts + 1
                    
    def getFormula(self, digits):
        formula = str(round(self.intercept, digits))
        for term in self.terms:
            formula = formula + term.getFormula(digits)
        return formula
        
    def saveRanges(self, x, fn):
        nrows = len(x)
        nvars = len(self.terms)
        values = np.zeros((nrows, nvars))
        for i in range(0, nrows):
            xrow = x[i]
            vrow = values[i]
            for t in range(0, len(self.terms)):
                term = self.terms[t]
                vrow[t] = term.value(xrow[t])

        with open(fn, 'w') as f:                
            for t in range(0, len(self.terms)):
                term = self.terms[t]
                mint = min(values[:,t])
                maxt = max(values[:,t])
                f.write(term.name + ' ' + str(mint) + ' ' + str(maxt) + '\n')            

    def saveRCSTerms(self, x, d):
        for t in range(0, len(self.terms)):            
            term = self.terms[t]
            if not term.isRCS: continue
            yvalues = []
            xmin = x[:,t].min()
            xmax = x[:,t].max()                
            xvalues = np.linspace(xmin, xmax, 100)
            for xt in xvalues:
                y = term.value(xt)
                yvalues += [y]
            fig, ax = plt.subplots()
            plt.plot(xvalues, yvalues)
            plt.xlabel(term.name, labelpad=20)
            plt.title('RCS term for ' + term.name)
            fig.savefig(os.path.join(d, 'rcs_' + term.name + '.pdf'))
                
    def loadTermsMICE(self, fn):
        rcsCoeffs = None;
        lines = []
        with open(fn) as ifn:    
            lines = ifn.readlines()

        pos = lines[0].index('est') + 2

        n = 1;
        while n < len(lines):
            line = lines[n]     
            n += 1
            
            s = line[0:pos].strip()
            
            v = s.split()
            if line[0] == ' ' or len(v) == 1: break
            valueStr = v[-1]
            value = float(valueStr)

            pos0 = s.index(valueStr)
            var = s[0:pos0].strip()

            if 'rcs' in var and var.index('rcs') == 0:
                pos1 = var.rfind(')')
                rcsString = var[4:pos1]
                pieces = rcsString.split('c')
                part1 = pieces[0].split(',')
                varName = part1[0].strip()
                rcsOrder = int(part1[1].strip())
                knotStr = pieces[1].replace("(", "").replace(")", "").split(",")
                rcsKnots = [float(k) for k in knotStr]
                coeffOrder = len(var) - len(var.replace("'", ""))
                
                if coeffOrder == 0:
                    rcsCoeffs = [0.0] * (rcsOrder - 1);
                if rcsCoeffs: 
                    rcsCoeffs[coeffOrder] = value;

                if coeffOrder == rcsOrder - 2:
                    term = RCSTerm(varName, rcsOrder, rcsCoeffs, rcsKnots)
                    self.addTerm(term)              
            else:
                if var == '(Intercept)':
                    self.setIntercept(value);
                else:
                    term = LinearTerm(var, value)
                    self.addTerm(term)

    def loadTermsGLM(self, fn):               
        rcsCoeffs = None;
        lines = []
        with open(fn) as ifn:    
            lines = ifn.readlines()

        reading = False
        n = 1;
        while n < len(lines):
            line = lines[n]
            n += 1

            if '(Intercept)' in line: 
                reading = True
                val = line.split()[1]
                pos = line.index(val) + len(val)
                
                # This breaks easily if file is not properly formatted:
                #pos = line.index('Estimate') + 8
                #continue
            
            if not reading: continue
            
            s = line[0:pos].strip()
            
            v = s.split()
            if line[0] == ' ' or len(v) == 1 or v[0] == '---': break   
            valueStr = v[-1]
            value = float(valueStr)

            pos0 = s.index(valueStr)
            var = s[0:pos0].strip()

            if 'rcs' in var and var.index('rcs') == 0:
                pos1 = var.rfind(')')
                rcsString = var[4:pos1]
                pieces = rcsString.split('c')
                part1 = pieces[0].split(',')
                varName = part1[0].strip()
                rcsOrder = int(part1[1].strip())
                knotStr = pieces[1].replace("(", "").replace(")", "").split(",")
                rcsKnots = [float(k) for k in knotStr]
                coeffOrder = len(var) - len(var.replace("'", ""))
                
                if coeffOrder == 0:
                    rcsCoeffs = [0.0] * (rcsOrder - 1);
                if rcsCoeffs: 
                    rcsCoeffs[coeffOrder] = value;

                if coeffOrder == rcsOrder - 2:
                    term = RCSTerm(varName, rcsOrder, rcsCoeffs, rcsKnots)
                    self.addTerm(term)              
            else:
                if var == '(Intercept)':
                    self.setIntercept(value);
                else:
                    term = LinearTerm(var, value)
                    self.addTerm(term)
                    
class ModelTerm(object):
    def __init__(self, name):
        self.isRCS = False
        self.name = name
        self.vtyp = 'float'
        self.coeffs = []
    def linearFeatures(self, x):
        return [0.0] * len(self.coeffs)
    def varRanges(self, x):
        # Scale coefficients by IQR (in floating-point variables) or
        # closest power-of-ten for integer variables.        
        if self.vtyp == 'category': 
            return [1]
        elif self.vtyp == 'int':
            n = np.floor(np.log10(max(x)))
            return [np.power(10, n)]
        elif self.vtyp == 'float':                                
            return [np.percentile(x, 75) - np.percentile(x, 25)]
    def getFormula(self, digits):
        return ''
    def varNames(self):
        return [self.name]
    def value(self, x): 
        return np.dot(self.coeffs, self.linearFeatures(x))
    
class LinearTerm(ModelTerm):
    def __init__(self, name, c):
        ModelTerm.__init__(self, name)
        self.coeffs = [c]

    def linearFeatures(self, x):
        return [x]

    def getFormula(self, digits):
        c = self.coeffs[0]
        sign = ' + ' if 0 < c else ' - '
        return sign + str(round(abs(c), digits)) + ' ' + self.name
    
    def __str__(self):
        res = "Linear term for " + self.name + "\n"
        res += "  Coefficient: " + str(self.coeffs[0])
        return res

class RCSTerm(ModelTerm):
    def __init__(self, name, k, c, kn):
        ModelTerm.__init__(self, name)
        self.isRCS = True        
        self.order = k
        self.coeffs = list(c)
        self.knots = list(kn)

    def cubic(self, u):
        t = np.maximum(0, u)
        return t * t * t
    
    def rcs(self, x, term):
        k = len(self.knots) - 1
        j = term - 1
        t = self.knots
        c = (t[k] - t[0]) * (t[k] - t[0])
        value = +self.cubic(x - t[j]) \
                -self.cubic(x - t[k - 1]) * (t[k] - t[j])/(t[k] - t[k-1]) \
                +self.cubic(x - t[k]) * (t[k - 1] - t[j])/(t[k] - t[k-1]) 
        return value / c
    
    def rcsform(self, term, digits):
        k = len(self.knots) - 1
        j = term - 1
        t = self.knots
        c = (t[k] - t[0]) * (t[k] - t[0])
          
        c0 = self.coeffs[term] / c
        sign0 = ' + ' if 0 < c0 else ' - '
        s = sign0 + str(round(abs(c0), digits[0])) + ' max(%s - ' + str(round(t[j], 3)) + ', 0)^3' 
    
        c1 = self.coeffs[term] * (t[k] - t[j])/(c * (t[k] - t[k-1]))    
        sign1 = ' - ' if 0 < c1 else ' + '
        s += sign1 + str(round(abs(c1), digits[1])) + ' max(%s - ' + str(round(t[k - 1], 3)) + ', 0)^3' 
    
        c2 = self.coeffs[term] * (t[k - 1] - t[j])/(c * (t[k] - t[k-1]))
        sign2 = ' + ' if 0 < c2 else ' - '        
        s += sign2 + str(round(c2, digits[2])) + ' max(%s - ' + str(round(t[k], 3)) + ', 0)^3' 
    
        return s

    def linearFeatures(self, x):
        feat = [0.0] * (self.order - 1)
        feat[0] = x
        for t in range(1, self.order - 1):
            feat[t] = self.rcs(x, t)
        return feat           

    def varRanges(self, x):
        rang = [0.0] * (self.order - 1)
        rang[0] = np.percentile(x, 75) - np.percentile(x, 25)
        for i in range(1, self.order - 1):
            y = self.rcs(x, i)
            rang[i] = np.percentile(y, 75) - np.percentile(y, 25)            
        return rang
    
    def varNames(self):
        nam = [''] * (self.order - 1)
        nam[0] = self.name
        for i in range(1, self.order - 1):
            nam[i] = self.name + ("'" * i)
        return nam
    
    def getFormula(self, digits):        
        c = self.coeffs[0]
        sign = ' + ' if 0 < c else ' - '
        s = sign + str(round(abs(c), digits)) + ' ' + self.name
        for i in range(1, self.order - 1):
            s = s + self.rcsform(i, [digits] * 3) % (self.name, self.name, self.name)
        return s
    
    def __str__(self):
        res = "RCS term of order " + str(self.order) + " for " + self.name + "\n"
        res += "  Coefficients:";
        for i in range(0, len(self.coeffs)):
            res += " " + str(self.coeffs[i])
        res += "\n"
        res += "  Knots:"
        for i in range(0, len(self.knots)):
            res += " " + str(self.knots[i])
        return res

In [ ]:
"""
Measurements inspired by Philip Tetlock's "Expert Political Judgment"
Equations take from Yaniv, Yates, & Smith (1991):
  "Measures of Descrimination Skill in Probabilistic Judgement"
"""

def calibration(outcome, prob, n_bins=10):
    """Calibration measurement for a set of predictions.
    When predicting events at a given probability, how far is frequency
    of positive outcomes from that probability?
    NOTE: Lower scores are better
    prob: array_like, float
        Probability estimates for a set of events
    outcome: array_like, bool
        If event predicted occurred
    n_bins: int
        Number of judgement categories to prefrom calculation over.
        Prediction are binned based on probability, since "discrete" 
        probabilities aren't required. 
    """
    prob = np.array(prob)
    outcome = np.array(outcome)

    c = 0.0
    # Construct bins
    judgement_bins = np.arange(n_bins + 1.0) / n_bins
    # Which bin is each prediction in?
    bin_num = np.digitize(prob,judgement_bins)
    for j_bin in np.unique(bin_num):
        # Is event in bin
        in_bin = bin_num == j_bin
        # Predicted probability taken as average of preds in bin
        predicted_prob = np.mean(prob[in_bin])
        # How often did events in this bin actually happen?
        true_bin_prob = np.mean(outcome[in_bin])
        # Squared distance between predicted and true times num of obs
        c += np.sum(in_bin) * ((predicted_prob - true_bin_prob) ** 2)
    return c / len(prob)


def calibration_table(outcome, prob, n_bins=10):
    """Calibration measurement for a set of predictions.
    When predicting events at a given probability, how far is frequency
    of positive outcomes from that probability?
    NOTE: Lower scores are better
    prob: array_like, float
        Probability estimates for a set of events
    outcome: array_like, bool
        If event predicted occurred
    n_bins: int
        Number of judgement categories to prefrom calculation over.
        Prediction are binned based on probability, since "discrete" 
        probabilities aren't required. 
    """
    prob = np.array(prob)
    outcome = np.array(outcome)

    c = 0.0
    # Construct bins
    judgement_bins = np.arange(n_bins + 1.0) / n_bins
    # Which bin is each prediction in?
    bin_num = np.digitize(prob, judgement_bins)

    counts = []
    true_prob = []
    pred_prob = []
    for j_bin in np.arange(n_bins + 1):
        # Is event in bin
        in_bin = bin_num == j_bin
#         # Predicted probability taken as average of preds in bin        
        predicted_prob = np.mean(prob[in_bin])
#         # How often did events in this bin actually happen?
        true_bin_prob = np.mean(outcome[in_bin])
        counts.append(np.sum(0 <= prob[in_bin]))
        true_prob.append(true_bin_prob) 
        pred_prob.append(predicted_prob)
    
    cal_table = pd.DataFrame({'pred_prob':pd.Series(np.array(pred_prob)), 
                              'count':pd.Series(np.array(counts)),
                              'true_prob':pd.Series(np.array(true_prob))}, 
                              columns=['pred_prob', 'count', 'true_prob'])
    cal_table.dropna(inplace=True)
    return cal_table 


def discrimination(outcome, prob, n_bins=10):
    """Discrimination measurement for a set of predictions.
    For each judgement category, how far from the base probability
    is the true frequency of that bin?
    NOTE: High scores are better
    prob: array_like, float
        Probability estimates for a set of events
    outcome: array_like, bool
        If event predicted occurred
    n_bins: int
        Number of judgement categories to prefrom calculation over.
        Prediction are binned based on probability, since "discrete" 
        probabilities aren't required. 
    """
    prob = np.array(prob)
    outcome = np.array(outcome)

    d = 0.0
    # Base frequency of outcomes
    base_prob = np.mean(outcome)
    # Construct bins
    judgement_bins = np.arange(n_bins + 1.0) / n_bins
    # Which bin is each prediction in?
    bin_num = np.digitize(prob,judgement_bins)
    for j_bin in np.unique(bin_num):
        in_bin = bin_num == j_bin
        true_bin_prob = np.mean(outcome[in_bin])
        # Squared distance between true and base times num of obs
        d += np.sum(in_bin) * ((true_bin_prob - base_prob) ** 2)
    return d / len(prob)

def caldis(outcome, probs, n_bins=10):
    c = calibration(outcome, probs, n_bins)
    d = discrimination(outcome, probs, n_bins)
    return c, d 

In [ ]:
sel_model = 1

if sel_model == 1:
    model_name = 'min'
if sel_model == 2:
    model_name = 'full'

In [ ]:
# Load IMC and Kenema data
imc_data_file = '../data/data.csv'
kenema_data_file = '../data/kenema/data.csv'

imc_data = pd.read_csv(imc_data_file, na_values="\\N")
kenema_data = pd.read_csv(kenema_data_file, na_values="\\N")

In [ ]:
# Compute transformation between viral load and CT:

min_ct = imc_data['cycletime'].min()
max_ct = imc_data['cycletime'].max()
min_log_pcr = kenema_data['PCR'].min()
max_log_pcr = kenema_data['PCR'].max()
print min_ct, max_log_pcr
print max_ct, min_log_pcr
b = (max_log_pcr - min_log_pcr) / (max_ct - min_ct)
a = min_log_pcr + b * max_ct
vl2ct_c1 = -1/b
vl2ct_c0 = +a/b
print 3*b
print vl2ct_c1, vl2ct_c0

# Compare with:
# Each 3-point decrease in Ct was associated with an ≈10-fold increase in Ebola viral load; 
# a Ct of 39 corresponded to ≈40 TCID50/mL and a Ct of 19 corresponded to ≈40 million TCID50/mL
# http://www.fda.gov/downloads/medicaldevices/safety/emergencysituations/ucm436313.pdf
# Based on this, 3*b should be close to 1

In [ ]:
# Generate datasets
test_data_folder = '../data/kenema/test'
test_data_file = '../data/kenema/test/all_data.csv'
if not os.path.exists(test_data_folder):
    os.makedirs(test_data_folder)
    
# Load imputation files for selected model, if any
from os import listdir, makedirs
from os.path import isfile, join, exists    
imp_data_folder = os.path.join(test_data_folder, model_name)
if not os.path.exists(imp_data_folder):
    os.makedirs(imp_data_folder)
imp_data_files = [join(imp_data_folder, f) for f in listdir(imp_data_folder) if isfile(join(imp_data_folder, f))]

if sel_model == 1:
    src_variables = ['OUT', 'PCR', 'AGE']
    variables = ['OUT', 'CT', 'AGE']
elif sel_model == 3:
    src_variables = ['OUT', 'PCR', 'AGE', 'DIARR', 'WEAK', 'JAUN', 'BNONE', 'TEMP', 'HEADCH', 'VOMIT', 'PABD']
    variables = ['OUT', 'CT', 'AGE', 'TEMP', 'HEADCH', 'BLEED', 'DIARR', 'JAUN', 'VOMIT', 'PABD', 'WEAK']
    
test_data = kenema_data[kenema_data['DIAG'] == 1][src_variables]

test_data['CT'] = vl2ct_c1 * test_data['PCR'] + vl2ct_c0

if 'SEX' in variables and 'GEND' in src_variables:
    test_data['SEX'] = 1 - test_data['GEND']
if 'BLEED' in variables and 'BNONE' in src_variables:
    test_data['BLEED'] = 1 - test_data['BNONE']
if 'JAUN' in variables:
    test_data['JAUN'] = 0 # all the non-missing values are 0, so MICE won't impute it

test_data = test_data[variables]
test_data.to_csv(test_data_file, index=False, na_rep="\\N")

test_data['OUT']

complete_data = test_data.dropna()
complete_data

### Results on complete data

In [ ]:
model_params = os.path.join(model_name, 'mice.txt')
model = LogRegModel(model_params)

x = complete_data[complete_data.columns[1:]].values
ytrue = [int(v) for v in complete_data[complete_data.columns[0]].values]
probs = model.predict(x)
ypred = [int(0.5 < p) for p in probs]

auc = roc_auc_score(ytrue, probs)
fpr, tpr, thresholds = roc_curve(ytrue, probs) 
brier = brier_score_loss(ytrue, probs)
cal, dis = caldis(ytrue, probs)
acc = accuracy_score(ytrue, ypred)
precision, recall, f1score, support = precision_recall_fscore_support(ytrue, ypred)

P = N = 0
TP = TN = 0
FP = FN = 0
for i in range(len(ytrue)):
    if ytrue[i] == 1:
        P += 1
        if ypred[i] == 1: TP += 1
        else: FN += 1
    else:
        N += 1
        if ypred[i] == 0: TN += 1
        else: FP += 1
            
sens = float(TP)/P
spec = float(TN)/N

# Positive and Negative Predictive Values
# https://en.wikipedia.org/wiki/Positive_and_negative_predictive_values
ppv = float(TP) / (TP + FP)
npv = float(TN) / (TN + FN)
        
# Likelihood ratios
# https://en.wikipedia.org/wiki/Likelihood_ratios_in_diagnostic_testing
lr_pos = sens / (1 - spec) if spec < 1 else np.inf
lr_neg = (1 - sens) / spec if 0 < spec else np.inf
    
# print "True outcomes:", ytrue
# print "Prediction   :", ypred
print "Number of cases :", len(ytrue)
print "Number of deaths:", np.sum(ytrue)
print "CFR             :", 100 * (float(np.sum(ytrue)) / len(ytrue))

print ""
print "Measures of performance"
print "AUC           :", auc
print "Brier         :", brier
print "Calibration   :", cal
print "Discrimination:", dis
print "Accuracy      :", acc
print "Sensitivity   :", sens
print "Specificity   :", spec
print "PPV           :", ppv
print "NPV           :", npv
print "LR+           :", lr_pos
print "LR-           :", lr_neg

# print "Precision (live)  :", precision[0]," (specificity for die)"
# print "Precision (die)   :", precision[1]," (specificity for live)"
# print "Sensitivity (live):", recall[0]
# print "Sensitivity (die) :", recall[1]
# print "F1 (live)         :", f1score[0]
# print "F1 (die)          :", f1score[1]

with open(os.path.join(model_name, 'kgh-comp.txt'), 'w') as of:
    of.write("Measures of performance\n")
    of.write("AUC           : " + str(auc) + "\n")
    of.write("Brier         : " + str(brier) + "\n")
    of.write("Calibration   : " + str(cal) + "\n")
    of.write("Discrimination: " + str(dis) + "\n")
    of.write("Accuracy      : " + str(acc) + "\n")    
    of.write("Sensitivity   : " + str(sens) + "\n")
    of.write("Specificity   : " + str(spec) + "\n")
    of.write("PPV           : " + str(ppv) + "\n")
    of.write("NPV           : " + str(npv) + "\n")
    of.write("LR+           : " + str(lr_pos) + "\n")
    of.write("LR-           : " + str(lr_neg) + "\n")    
    
#     of.write("Precision (live)  : " + str(precision[0]) + " (specificity for die)\n")
#     of.write("Precision (die)   : " + str(precision[1]) + " (specificity for live)\n")
#     of.write("Sensitivity (live): " + str(recall[0]) + "\n")
#     of.write("Sensitivity (die) : " + str(recall[1]) + "\n")
#     of.write("F1 (live)         : " + str(f1score[0]) + "\n")
#     of.write("F1 (die)          : " + str(f1score[1]) + "\n")

fig, ax = plt.subplots()
plt.xlim([-0.1, 1.1])
plt.ylim([-0.1, 1.1])
plt.plot([0, 1], [0, 1], 'k--', c='grey')
plt.plot(fpr, tpr, color='black')
plt.xlabel('1 - Specificity')
plt.ylabel('Sensitivity')
fig.savefig(os.path.join(model_name, 'kenema-roc-complete.pdf'))

cal_table = calibration_table(ytrue, probs, 10)
fig, ax = plt.subplots()
plt.plot([0.05, 0.95], [0.05, 0.95], '-', c='grey', linewidth=0.5, zorder=1)
plt.xlim([-0.1, 1.1])
plt.ylim([-0.1, 1.1])
plt.xlabel('Predicted Risk')
plt.ylabel('Observed Risk')
x = cal_table['pred_prob']
y = cal_table['true_prob']
# f = interp1d(x, y, kind='cubic')
# xnew = np.linspace(min(x), max(x), num=50, endpoint=True)    
# plt.plot(xnew, f(xnew))
plt.plot(x, y, color='black')
fig.savefig(os.path.join(model_name, 'kenema-cal-complete.pdf'))

### Resuls on entire dataset

Need to impute, as there is missing data, but can skip and move on to graphs if already run imputation earlier (imputed files are stored)

In [ ]:
%%R -i imp_data_folder,test_data_file -o imp_data_files

# Imputation in R using MICE
library(mice)

num_imp <- 50

src_data <- read.table(test_data_file, sep=",", header=TRUE, na.strings="\\N")

imp_data <- mice(src_data, meth='pmm', m=num_imp)
var_drop <- c(".imp", ".id")
imp_data_files <- character(0)
for (iter in 1:num_imp) {
    comp_data <- complete(imp_data, action=iter)  
    comp_data <- comp_data[,!(names(comp_data) %in% var_drop)]
    fn <- paste(imp_data_folder, "/imputation-", iter, ".csv", sep="")
    write.csv(comp_data, file=fn, row.names=FALSE)
    imp_data_files <- c(imp_data_files, fn)
}

In [ ]:
ytrue_all = []
probs_all = []

cfr_list = []
auc_list = [] 
brier_list = [] 
cal_list = [] 
dis_list = [] 
acc_list = []
sens_list = [] 
spec_list = []
ppv_list = [] 
npv_list = []
lr_pos_list = []
lr_neg_list = []

for fn in imp_data_files:
    data = pd.read_csv(fn)

    x = data[data.columns[1:]].values
    ytrue = [int(v) for v in data[data.columns[0]].values]
    probs = list(model.predict(x))
    
    ypred = [int(0.5 < p) for p in probs]
    cfr = float(np.sum(ytrue)) / len(ytrue)
    
    ytrue_all += ytrue
    probs_all += probs
    
    P = N = 0
    TP = TN = 0
    FP = FN = 0
    for i in range(len(ytrue)):
        if ytrue[i] == 1:
            P += 1
            if ypred[i] == 1: TP += 1
            else: FN += 1
        else:
            N += 1
            if ypred[i] == 0: TN += 1
            else: FP += 1
            
    sens = float(TP)/P
    spec = float(TN)/N

    ppv = float(TP) / (TP + FP)
    npv = float(TN) / (TN + FN)
    
    lr_pos = sens / (1 - spec) if spec < 1 else np.inf
    lr_neg = (1 - sens) / spec if 0 < spec else np.inf    
    
    auc = roc_auc_score(ytrue, probs)
    brier = brier_score_loss(ytrue, probs)
    cal, dis = caldis(ytrue, probs)
    acc = accuracy_score(ytrue, ypred)
    
    cfr_list.append(cfr)
    auc_list.append(auc)
    brier_list.append(brier)
    cal_list.append(cal)
    dis_list.append(dis)
    acc_list.append(acc)
    sens_list.append(sens)
    spec_list.append(spec)
    ppv_list.append(ppv)
    npv_list.append(npv)
    lr_pos_list.append(lr_pos)
    lr_neg_list.append(lr_neg)    
    
#     prec0_list.append(precision[0])
#     prec1_list.append(precision[1])
#     rec0_list.append(recall[0])
#     rec1_list.append(recall[1])
#     f10_list.append(f1score[0])
#     f11_list.append(f1score[1])  

cfr_mean = np.mean(cfr_list)
auc_mean = np.mean(auc_list)
brier_mean = np.mean(brier_list)
cal_mean = np.mean(cal_list)
dis_mean = np.mean(dis_list)      
acc_mean = np.mean(acc_list)
sens_mean = np.mean(sens_list)      
spec_mean = np.mean(spec_list)
ppv_mean = np.mean(ppv_list)
npv_mean = np.mean(npv_list)
lr_pos_mean = np.mean(lr_pos_list)
lr_neg_mean = np.mean(lr_neg_list)

# prec0_mean = np.mean(prec0_list)
# prec1_mean = np.mean(prec1_list)
# rec0_mean = np.mean(rec0_list)
# rec1_mean = np.mean(rec1_list)
# f10_mean = np.mean(f10_list)
# f11_mean = np.mean(f11_list)
 
cfr_dev = np.std(cfr_list)
auc_dev = np.std(auc_list)
brier_dev = np.std(brier_list)
cal_dev = np.std(cal_list)
dis_dev = np.std(dis_list)      
acc_dev = np.std(acc_list)
sens_dev = np.std(sens_list)      
spec_dev = np.std(spec_list)
ppv_dev = np.std(ppv_list)
npv_dev = np.std(npv_list)
lr_pos_dev = np.std(lr_pos_list)
lr_neg_dev = np.std(lr_neg_list)

# prec0_dev = np.std(prec0_list)
# prec1_dev = np.std(prec1_list)
# rec0_dev = np.std(rec0_list)
# rec1_dev = np.std(rec1_list)
# f10_dev = np.std(f10_list)
# f11_dev = np.std(f11_list)    
    
print "Number of cases :", len(ytrue)
print "Mean CFR        :", 100 * cfr_mean
print ""
print "Measures of performance"
print "AUC           :", auc_mean, '+/-', auc_dev
print "Brier         :", brier_mean, '+/-', brier_dev
print "Calibration   :", cal_mean, '+/-', cal_dev
print "Discrimination:", dis_mean, '+/-', dis_dev
print "Accuracy      :", acc_mean, '+/-', acc_dev
print "Sensitivity   :", sens_mean, '+/-', sens_dev
print "Specificity   :", spec_mean, '+/-', spec_dev
print "PPV           :", ppv_mean, '+/-', ppv_dev
print "NPV           :", npv_mean, '+/-', npv_dev
print "LR+           :", lr_pos_mean, '+/-', lr_pos_dev
print "LR-           :", lr_neg_mean, '+/-', lr_neg_dev


# print "Precision (live)  :", prec0_mean, '+/-', prec0_dev," (specificity for die)"
# print "Precision (die)   :", prec1_mean, '+/-', prec1_dev," (specificity for live)"
# print "Sensitivity (live):", rec0_mean, '+/-', rec0_dev
# print "Sensitivity (die) :", rec1_mean, '+/-', rec1_dev
# print "F1 (live)         :", f10_mean, '+/-', f10_dev
# print "F1 (die)          :", f11_mean, '+/-', f11_dev

with open(os.path.join(model_name, 'kgh-imp.txt'), 'w') as of:
    of.write("Measures of performance\n")
    of.write("AUC           : " + str(auc_mean) + "+/-" + str(auc_dev) + "\n")
    of.write("Brier         : " + str(brier_mean) + "+/-" + str(brier_dev) + "\n")
    of.write("Calibration   : " + str(cal_mean) + "+/-" + str(cal_dev) + "\n")
    of.write("Discrimination: " + str(dis_mean) + "+/-" + str(dis_dev) + "\n")
    of.write("Accuracy      : " + str(acc_mean) + "+/-" + str(acc_dev) + "\n")
    of.write("Sensitivity   : " + str(sens_mean) + "+/-" + str(sens_dev) + "\n")
    of.write("Specificity   : " + str(spec_mean) + "+/-" + str(spec_dev) + "\n")    
    of.write("PPV           : " + str(ppv_mean) + "+/-" + str(ppv_dev) + "\n")
    of.write("NPV           : " + str(npv_mean) + "+/-" + str(npv_dev) + "\n")     
    of.write("LR+           : " + str(lr_pos_mean) + "+/-" + str(lr_pos_dev) + "\n")
    of.write("LR-           : " + str(lr_neg_mean) + "+/-" + str(lr_neg_dev) + "\n") 

#     of.write("Precision (live)  : " + str(prec0_mean) + "+/-" + str(prec0_dev) + " (specificity for die)\n")
#     of.write("Precision (die)   : " + str(prec1_mean) + "+/-" + str(prec1_dev) + " (specificity for live)\n")
#     of.write("Sensitivity (live): " + str(rec0_mean) + "+/-" + str(rec0_dev) + "\n")
#     of.write("Sensitivity (die) : " + str(rec1_mean) + "+/-" + str(rec1_dev) + "\n")
#     of.write("F1 (live)         : " + str(f10_mean) + "+/-" + str(f10_dev) + "\n")
#     of.write("F1 (die)          : " + str(f11_mean) + "+/-" + str(f11_dev) + "\n")

In [ ]:
# ROC curve

fig, ax = plt.subplots()
plt.xlim([-0.2, 1.1])
plt.ylim([-0.1, 1.1])
plt.plot([0, 1], [0, 1], 'k--', c='grey', linewidth=0.5)
plt.xlabel('1 - Specificity')
plt.ylabel('Sensitivity')

imp_fpr = []
imp_tpr = []

for fn in imp_data_files:
    data = pd.read_csv(fn)
    
    x = data[data.columns[1:]].values
    ytrue = [int(v) for v in data[data.columns[0]].values]
    probs = list(model.predict(x))
    
    # Drawing the ROC from each imputed dataset
    fpr, tpr, thresholds = roc_curve(ytrue, probs) 
    plt.plot(fpr, tpr, color='black', alpha=0.05)
    imp_fpr += [fpr]
    imp_tpr += [tpr]
    
# Macro-average of ROC cuve over all imputations.

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate(imp_fpr))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(0, len(imp_fpr)):
    mean_tpr += interp(all_fpr, imp_fpr[i], imp_tpr[i])
mean_tpr /= len(imp_fpr)

plt.plot(all_fpr, mean_tpr, color='red', alpha=1.0)
fig.savefig(os.path.join(model_name, 'kenema-roc-imputed.pdf'))

In [ ]:
# Calibration curve

smooth = False

fig, ax = plt.subplots()
plt.plot([0.05, 0.95], [0.05, 0.95], '-', c='grey', linewidth=0.5, zorder=1)
plt.xlim([-0.1, 1.1])
plt.ylim([-0.1, 1.1])
plt.xlabel('Predicted Risk')
plt.ylabel('Observed Risk')

imp_ppr = []
imp_tpr = []

for fn in imp_data_files:
    data = pd.read_csv(fn)
    
    x = data[data.columns[1:]].values
    ytrue = [int(v) for v in data[data.columns[0]].values]
    probs = list(model.predict(x))
    
    # Drawing the calibration from each imputed dataset
    cal_table = calibration_table(ytrue, probs, 10)

    x = cal_table['pred_prob']
    y = cal_table['true_prob']
    if smooth:
        f = interp1d(x, y, kind='cubic')
        xnew = np.linspace(min(x), max(x), num=50, endpoint=True)    
        plt.plot(xnew, f(xnew), color='black', alpha=0.1)    
    else:    
        plt.plot(x, y, color='black', alpha=0.1)
    imp_ppr += [x]
    imp_tpr += [y]
    
all_ppr = np.unique(np.concatenate(imp_ppr))

mean_tpr = np.zeros_like(all_ppr)
for i in range(0, len(imp_ppr)):
    mean_tpr += interp(all_ppr, imp_ppr[i], imp_tpr[i])
mean_tpr /= len(imp_ppr)

if smooth:
    xnew = np.linspace(min(all_ppr), max(all_ppr), num=2 * len(all_ppr), endpoint=True)    
    f = interp1d(all_ppr, mean_tpr, kind='cubic')    
    plt.plot(xnew, f(xnew), color='red', alpha=1.0)
else:
    plt.plot(all_ppr, mean_tpr, color='red', alpha=1.0)

fig.savefig(os.path.join(model_name, 'kenema-cal-imputed.pdf'))